In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
from scipy import stats
import numpy as np
import requests
import pandas as pd
import json
from citipy import citipy
import random

# Import API key
from config import api_key

In [3]:
com_restaurants = pd.read_csv("cafes_and_restaurants.csv")
com_restaurants

,Census year,Block ID,Property ID,Base property ID,Street address,CLUE small area,Trading name,Industry (ANZSIC4) code,Industry (ANZSIC4) description,Seating type,Number of seats,x coordinate,y coordinate,Location
0,2002,203,109358,109358,832-834 Swanston Street CARLTON 3053,Carlton,Unibite,4512,Takeaway Food Services,Seats - Indoor,30,144.96490,-37.79670,POINT (144.9649 -37.7967)
1,2002,204,101613,101613,377-391 Cardigan Street CARLTON 3053,Carlton,Clyde Hotel,4520,"Pubs, Taverns and Bars",Seats - Indoor,50,144.96580,-37.79730,POINT (144.9658 -37.7973)
2,2002,205,104469,104469,114-152 Grattan Street CARLTON 3053,Carlton,Royal Women's Hospital Coffee Shop,4512,Takeaway Food Services,Seats - Indoor,25,144.96480,-37.80000,POINT (144.9648 -37.8)
3,2002,206,103691,103691,221-229 Faraday Street CARLTON 3053,Carlton,Thresherman's Bakehouse,4511,Cafes and Restaurants,Seats - Indoor,70,144.96670,-37.79890,POINT (144.9667 -37.7989)
4,2002,206,103691,103691,221-229 Faraday Street CARLTON 3053,Carlton,Thresherman's Bakehouse,4511,Cafes and Restaurants,Seats - Outdoor,42,144.96670,-37.79890,POINT (144.9667 -37.7989)
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
50647,2019,861,102507,102507,185-189 Domain Road SOUTH YARRA VIC 3141,South Yarra,The Golden Triangle Restaurant,4511,Cafes and Restaurants,Seats - Indoor,60,144.98226,-37.83434,POINT (144.9822639 -37.83434411)
50648,2019,862,102498,102498,131-133 Domain Road SOUTH YARRA VIC 3141,South Yarra,Entrecote - Paris Steakhouse & Winebar,4511,Cafes and Restaurants,Seats - Indoor,120,144.98049,-37.83413,POINT (144.980491 -37.83412514)
50649,2019,862,102498,102498,131-133 Domain Road SOUTH YARRA VIC 3141,South Yarra,Entrecote - Paris Steakhouse & Winebar,4511,Cafes and Restaurants,Seats - Outdoor,30,144.98049,-37.83413,POINT (144.980491 -37.83412514)
50650,2019,862,109532,109532,2-14 Hope Street SOUTH YARRA VIC 3141,South Yarra,Bistro Gitan,4511,Cafes and Restaurants,Seats - Outdoor,6,144.97931,-37.83714,POINT (144.9793101 -37.83713828)


In [4]:
sensors_df = pd.read_csv("PCS_Locations.csv")
sensors_df

,sensor_id,sensor_description,sensor_name,installation_date,status,note,direction_1,direction_2,latitude,longitude,location
0,55,Elizabeth St-La Trobe St (East),Eli380_T,2018/07/19,A,NaN,North,South,-37.809889,144.961343,"(-37.80988941, 144.96134331)"
1,45,Little Collins St-Swanston St (East),Swa148_T,2017/06/29,A,NaN,North,South,-37.814141,144.966094,"(-37.81414074, 144.9660938)"
2,61,RMIT Building 14,RMIT14_T,2019/06/28,A,NaN,North,South,-37.807675,144.963091,"(-37.80767455, 144.96309115)"
3,7,Birrarung Marr,Fed_T,2014/12/17,A,NaN,East,West,-37.818629,144.971694,"(-37.81862929, 144.97169395)"
4,72,Flinders St- ACMI,ACMI_T,2020/11/30,A,NaN,East,West,-37.817263,144.968728,"(-37.81726337, 144.96872809)"
...,...,...,...,...,...,...,...,...,...,...,...
69,51,QVM-Franklin St (North),Fra118_T,2017/11/30,A,NaN,West,East,-37.808418,144.959063,"(-37.80841814, 144.95906317)"
70,10,Victoria Point,BouHbr_T,2009/04/23,A,NaN,West,East,-37.818765,144.947105,"(-37.81876473, 144.94710545)"
71,43,Monash Rd-Swanston St (West),UM2_T,2015/04/15,A,NaN,South,North,-37.798445,144.964118,"(-37.79844525, 144.96411782)"
72,73,Bourke St - Spencer St (South),Bou655_T,2020/10/02,A,NaN,West,East,-37.816957,144.954154,"(-37.81695684, 144.95415373)"


In [5]:
sensors_df["sensor_description"].unique()

array(['Elizabeth St-La Trobe St (East)',
       'Little Collins St-Swanston St (East)', 'RMIT Building 14',
       'Birrarung Marr', 'Flinders St- ACMI', 'La Trobe St (North)',
       '231 Bourke St', 'Sandridge Bridge',
       'Flinders La-Swanston St (West)',
       'Melbourne Convention Exhibition Centre',
       'Flinders La - Swanston St (West) Temporary',
       'Royal Pde - Grattan St', 'Alfred Place', 'Collins Place (South)',
       'Swanston St - City Square', 'State Library - New',
       'State Library', 'Flinders Ln -Degraves St (South)',
       'Melbourne Central-Elizabeth St (East)', 'Lincoln-Swanston (West)',
       'Flinders Ln -Degraves St (North)',
       'Flinders Ln -Degraves St (Crossing)', 'Melbourne Central',
       'St Kilda Rd-Alexandra Gardens', 'Errol Street (East)',
       'Westwood Place', 'Bourke St-Russell St (West)',
       'Spencer St-Collins St (South)', 'Flinders St-Spring St (West)',
       'Flagstaff Station', 'New Quay', 'Collins Street (North)',


In [7]:
# Limit sensors to just CBD using the following coordinates: -37.819738, 144.952987; -37.805909, 144.975164
sensors_reduced = sensors_df.copy()
sensors_reduced = sensors_reduced.loc[(sensors_reduced["latitude"] >= -37.819738) & (sensors_reduced["latitude"] <= -37.805909) & 
               (sensors_reduced["longitude"] >= 144.952987) & (sensors_reduced["longitude"] <= 144.975164), :]

# Only use the following 12 sensors:
sensors_reduced = sensors_reduced.loc[(sensors_reduced["sensor_description"] == "Bourke Street Mall (North)") | 
                                        (sensors_reduced["sensor_description"] == "Collins Place (South)") |
                                        (sensors_reduced["sensor_description"] == "Lonsdale St-Spring St (West)") |
                                        (sensors_reduced["sensor_description"] == "Collins Street (North)") |
                                        (sensors_reduced["sensor_description"] == "Flinders St-Elizabeth St (East)") | 
                                        (sensors_reduced["sensor_description"] == "QV Market-Peel St") | 
                                        (sensors_reduced["sensor_description"] == "Queen St (West)") |
                                        (sensors_reduced["sensor_description"] == "Melbourne Central-Elizabeth St (East)") |
                                        (sensors_reduced["sensor_description"] == "QVM-Franklin St (North)") |
                                        (sensors_reduced["sensor_description"] == "Bourke St - Spencer St (North)") |
                                        (sensors_reduced["sensor_description"] == "RMIT Building 14") |
                                        (sensors_reduced["sensor_description"] == "Spring St- Flinders st (West)")
                                         ,:]
sensors_reduced[["sensor_id","sensor_description"]]

,sensor_id,sensor_description
2,61,RMIT Building 14
13,17,Collins Place (South)
18,47,Melbourne Central-Elizabeth St (East)
31,53,Collins Street (North)
46,1,Bourke Street Mall (North)
51,40,Lonsdale St-Spring St (West)
55,36,Queen St (West)
56,58,Bourke St - Spencer St (North)
63,22,Flinders St-Elizabeth St (East)
65,27,QV Market-Peel St


In [29]:
# Base URL required for API call
url = "https://developers.zomato.com/api/v2.1/search?"

# Parameters to run through Query URL
start = np.arange(0,40,20)
count = 20
city = 259
radius = 50
b = 0

In [2]:
# Latitude and Longitudes of Key Locations:
# Cnr William & Bourke; Franklin & Elizabeth; GPO; Russell & Collins; 
# (Optional: Elizabeth & Flinders)
#lat = [-37.8156,-37.8081,-37.8136,-37.8147]
#lon = [144.9581,144.9604,144.9631,144.9689]

In [33]:
# Latitude and Longitudes of Sensor Locations:
lat = list(sensors_reduced["latitude"])
lon = list(sensors_reduced["longitude"])

In [34]:
# Empty list to hold all API calls
response_data = []

In [35]:
# Loop through Zomato API
for a in range(len(start)):
    for b in range(len(lat)):
        query_url = f"{url}entity_id={city}&entity_type=metro&lat={lat[b]}&lon={lon[b]}&radius={radius}&sort=real_distance&apikey={api_key}&start={start[a]}&count={count}"
        response_data.append(requests.get(query_url).json())
        print(start[a], lat[b], lon[b])
    b += 1

0 -37.80767455 144.96309115
0 -37.81362542 144.97323592
0 -37.81160331 144.96220077
0 -37.8156419 144.965499
0 -37.813494399999996 144.96515324
0 -37.80999341 144.97227588
0 -37.81652527 144.96121063
0 -37.81686074 144.95358075
0 -37.81786445 144.96506823
0 -37.80606888 144.95644693
0 -37.80841814 144.95906317
0 -37.81515275 144.97467662
20 -37.80767455 144.96309115
20 -37.81362542 144.97323592
20 -37.81160331 144.96220077
20 -37.8156419 144.965499
20 -37.813494399999996 144.96515324
20 -37.80999341 144.97227588
20 -37.81652527 144.96121063
20 -37.81686074 144.95358075
20 -37.81786445 144.96506823
20 -37.80606888 144.95644693
20 -37.80841814 144.95906317
20 -37.81515275 144.97467662


In [120]:
# Empty list to hold all restaurant names, aggregate rating, hours of operation, type of eatery, price 
restr_names = []
rating = []
cuisine_type = []
price_range = []
restr_lat = []
restr_lon = []
sensor_loc = []
sensor_id = []
c = 0
d = 0
# Populate lists with JSON data from Zomato 
for c in range(len(response_data)):     
    if c == 12:
        d = 0
    else:
        d = d
    for x in range(count):
        restr_names.append(response_data[c]["restaurants"][x]["restaurant"]["name"])
        rating.append(response_data[c]["restaurants"][x]["restaurant"]["user_rating"]["aggregate_rating"])
        cuisine_type.append(response_data[c]["restaurants"][x]["restaurant"]["cuisines"])
        price_range.append(response_data[c]["restaurants"][x]["restaurant"]["price_range"])
        restr_lat.append(response_data[c]["restaurants"][x]["restaurant"]["location"]["latitude"])
        restr_lon.append(response_data[c]["restaurants"][x]["restaurant"]["location"]["longitude"])
        sensor_id.append(list(sensors_reduced["sensor_id"])[d])
        sensor_loc.append(list(sensors_reduced["sensor_description"])[d])
        print(c,x,d)
    if c < 12:
        d += 1 
    elif c >= 12:
        d += 1  
    c += 1

0 0 0
0 1 0
0 2 0
0 3 0
0 4 0
0 5 0
0 6 0
0 7 0
0 8 0
0 9 0
0 10 0
0 11 0
0 12 0
0 13 0
0 14 0
0 15 0
0 16 0
0 17 0
0 18 0
0 19 0
1 0 1
1 1 1
1 2 1
1 3 1
1 4 1
1 5 1
1 6 1
1 7 1
1 8 1
1 9 1
1 10 1
1 11 1
1 12 1
1 13 1
1 14 1
1 15 1
1 16 1
1 17 1
1 18 1
1 19 1
2 0 2
2 1 2
2 2 2
2 3 2
2 4 2
2 5 2
2 6 2
2 7 2
2 8 2
2 9 2
2 10 2
2 11 2
2 12 2
2 13 2
2 14 2
2 15 2
2 16 2
2 17 2
2 18 2
2 19 2
3 0 3
3 1 3
3 2 3
3 3 3
3 4 3
3 5 3
3 6 3
3 7 3
3 8 3
3 9 3
3 10 3
3 11 3
3 12 3
3 13 3
3 14 3
3 15 3
3 16 3
3 17 3
3 18 3
3 19 3
4 0 4
4 1 4
4 2 4
4 3 4
4 4 4
4 5 4
4 6 4
4 7 4
4 8 4
4 9 4
4 10 4
4 11 4
4 12 4
4 13 4
4 14 4
4 15 4
4 16 4
4 17 4
4 18 4
4 19 4
5 0 5
5 1 5
5 2 5
5 3 5
5 4 5
5 5 5
5 6 5
5 7 5
5 8 5
5 9 5
5 10 5
5 11 5
5 12 5
5 13 5
5 14 5
5 15 5
5 16 5
5 17 5
5 18 5
5 19 5
6 0 6
6 1 6
6 2 6
6 3 6
6 4 6
6 5 6
6 6 6
6 7 6
6 8 6
6 9 6
6 10 6
6 11 6
6 12 6
6 13 6
6 14 6
6 15 6
6 16 6
6 17 6
6 18 6
6 19 6
7 0 7
7 1 7
7 2 7
7 3 7
7 4 7
7 5 7
7 6 7
7 7 7
7 8 7
7 9 7
7 10 7
7 11 7
7 12 7
7 13 7
7 

In [121]:
len(sensor_id)

480

In [37]:
len(restr_names)

400

In [188]:
response_data[0]["restaurants"]

[{'restaurant': {'R': {'res_id': 17849428,
    'is_grocery_store': False,
    'has_menu_status': {'delivery': -1, 'takeaway': -1}},
   'apikey': '4ceef556bc96bbbc555b4f3b3ce337d3',
   'id': '17849428',
   'name': 'Doughboys Doughnuts',
   'url': 'https://www.zomato.com/melbourne/doughboys-doughnuts-cbd?utm_source=api_basic_user&utm_medium=api&utm_campaign=v2.1',
   'location': {'address': '535 Bourke Street, CBD, Melbourne',
    'locality': 'CBD',
    'city': 'Melbourne',
    'city_id': 259,
    'latitude': '-37.8159507485',
    'longitude': '144.9578592181',
    'zipcode': '3000',
    'country_id': 14,
    'locality_verbose': 'CBD, Melbourne'},
   'switch_to_order_menu': 0,
   'cuisines': 'Coffee and Tea, Desserts',
   'timings': '7 AM to 4:30 PM (Mon-Fri), 9 AM to 4 PM (Sat), Sun Closed',
   'average_cost_for_two': 20,
   'price_range': 1,
   'currency': '$',
   'highlights': ['Cash',
    'Debit Card',
    'Delivery',
    'No Alcohol Available',
    'Credit Card',
    'Takeaway Avail

In [123]:
# Make DataFrame from list data
zomato_df = pd.DataFrame({
    "Name": restr_names,
    "Aggregate Rating": rating,
    "Type": cuisine_type,
    "Price": price_range,
    "Latitude": restr_lat,
    "Longitude": restr_lon,
    "Sensor ID": sensor_id,
    "Sensor": sensor_loc
})

zomato_df.tail(10)

,Name,Aggregate Rating,Type,Price,Latitude,Longitude,Sensor ID,Sensor
470,Subway,0,Sandwich,2,-37.8152366746,144.9721745029,75,Spring St- Flinders st (West)
471,Top Juice,3.4,"Beverages, Healthy Food, Salad",2,-37.8145318643,144.9722610042,75,Spring St- Flinders st (West)
472,Café Alcaston,4.2,"Coffee and Tea, Cafe Food",2,-37.8132853850,144.9736530706,75,Spring St- Flinders st (West)
473,Bean Republic,3.3,"Coffee and Tea, Sandwich, Cafe Food",1,-37.8143483106,144.9722868204,75,Spring St- Flinders st (West)
474,Market Lane Coffee,3.5,Coffee and Tea,2,-37.8133629926,144.9734053016,75,Spring St- Flinders st (West)
475,La Belle Miette,4.2,"Desserts, French, Coffee and Tea",1,-37.8133857716,144.9733171240,75,Spring St- Flinders st (West)
476,Petit Cafe,0,"Coffee and Tea, Cafe Food",1,-37.8157658739,144.9721242115,75,Spring St- Flinders st (West)
477,The Fair Trader,3.9,"Coffee and Tea, Cafe Food",3,-37.8155513341,144.9720055237,75,Spring St- Flinders st (West)
478,Vacation Coffee,4.1,Coffee and Tea,2,-37.8159594889,144.9721560627,75,Spring St- Flinders st (West)
479,Sargon,2.9,"Coffee and Tea, Cafe Food",2,-37.8135219158,144.9728990346,75,Spring St- Flinders st (West)


In [124]:
# Drop duplicates from DataFrame
z_clean = zomato_df.drop_duplicates(subset="Name", keep=False).copy()
z_clean.count()

Name                371
Aggregate Rating    371
Type                371
Price               371
Latitude            371
Longitude           371
Sensor ID           371
Sensor              371
dtype: int64

In [125]:
# Change column types of Rating, Latitude and Longitude from strings to numbers (float)
z_clean["Aggregate Rating"] = z_clean["Aggregate Rating"].astype(float)
z_clean["Latitude"] = z_clean["Latitude"].astype(float)
z_clean["Longitude"] = z_clean["Longitude"].astype(float)
z_clean

,Name,Aggregate Rating,Type,Price,Latitude,Longitude,Sensor ID,Sensor
0,Little Bang Espresso,3.6,Coffee and Tea,1,-37.807932,144.963255,61,RMIT Building 14
1,The College Dropout,3.9,"Burger, Desserts",2,-37.807741,144.963571,61,RMIT Building 14
2,Poolhouse Coffee,3.7,"Desserts, Cafe Food",2,-37.807249,144.963121,61,RMIT Building 14
3,Standing Room,3.5,"Coffee and Tea, Cafe Food",2,-37.808027,144.963410,61,RMIT Building 14
4,Simply Desi Canteen,0.0,Indian,1,-37.807741,144.963711,61,RMIT Building 14
5,Suzy Lee,0.0,"Dumplings, Japanese, Sushi",2,-37.807453,144.963667,61,RMIT Building 14
6,Canto Cafe,0.0,"Pizza, Sandwich",2,-37.807522,144.963708,61,RMIT Building 14
7,Chippie,0.0,Fast Food,1,-37.807685,144.963789,61,RMIT Building 14
8,Holy Guacamoly,0.0,Mexican,1,-37.807737,144.963807,61,RMIT Building 14
9,The Vine Coffee Roasters,3.5,"Coffee and Tea, Sandwich",2,-37.808308,144.963056,61,RMIT Building 14


In [51]:
# List all unique eatery types for further reduction into broader categories
pd.set_option('display.max_rows', None)
list(z_clean["Type"].sort_values().unique())

['African, Bar Food',
 'African, Contemporary',
 'American, BBQ',
 'American, BBQ, Sandwich',
 'American, Burger',
 'American, Cafe Food, Coffee and Tea',
 'American, Desserts',
 'American, Grill',
 'Asian',
 'Asian Fusion, Pan Asian',
 'Asian, BBQ, Chinese',
 'Asian, Desserts, Ice Cream',
 'Asian, Fusion',
 'Asian, Indonesian',
 'Asian, Japanese',
 'Asian, Vietnamese',
 'Australian, Contemporary',
 'BBQ, American',
 'BBQ, Brazilian, Steak',
 'Bakery',
 'Bakery, Coffee and Tea',
 'Bakery, Coffee and Tea, Sandwich',
 'Bakery, Desserts',
 'Bakery, Middle Eastern',
 'Bakery, Patisserie, Coffee and Tea',
 'Bakery, Sandwich',
 'Bar Food',
 'Bar Food, Modern Australian',
 'Beverages',
 'Beverages, Bubble Tea',
 'British, Pub Food',
 'Bubble Tea',
 'Bubble Tea, Asian',
 'Bubble Tea, Asian, Tea',
 'Bubble Tea, Beverages',
 'Bubble Tea, Desserts',
 'Bubble Tea, Taiwanese',
 'Bubble Tea, Tea',
 'Bubble Tea, Tea, Beverages',
 'Burger',
 'Burger, Bar Food',
 'Burger, Desserts',
 'Cafe Food',
 'Caf

In [126]:
# Reduce eateries to the following types: 
# restaurant (regardless of ethnicity), cafe, beverages (including bubble tea, juices, etc.), 
# bar, bakery, desserts, pub, fast food (takeaway),

z_clean_reduced = z_clean.replace({'African, Bar Food': 'Bar',
    'African, Contemporary': 'Restaurant',
    'American, BBQ': 'Restaurant',
    'American, BBQ, Sandwich': 'Cafe',
    'American, Burger': 'Restaurant',
    'American, Cafe Food, Coffee and Tea': 'Cafe',
    'American, Desserts': 'Desserts',
    'American, Grill': 'Restaurant',
    'Asian': 'Restaurant',
    'Asian Fusion, Pan Asian': 'Restaurant',
    'Asian, BBQ, Chinese': 'Restaurant',
    'Asian, Desserts, Ice Cream': 'Restaurant',
    'Asian, Fusion': 'Restaurant',
    'Asian, Indonesian': 'Restaurant',
    'Asian, Japanese': 'Restaurant',
    'Asian, Vietnamese': 'Restaurant',
    'Australian, Contemporary': 'Restaurant',
    'BBQ, American': 'Restaurant',
    'BBQ, Brazilian, Steak': 'Restaurant',
    'Bakery, Coffee and Tea': 'Cafe',
    'Bakery, Coffee and Tea, Sandwich': 'Cafe',
    'Bakery, Desserts': 'Bakery',
    'Bakery, Middle Eastern': 'Bakery',
    'Bakery, Patisserie, Coffee and Tea': 'Bakery',
    'Bakery, Sandwich': 'Bakery',
    'Bar Food': 'Bar',
    'Bar Food, Modern Australian': 'Bar',
    'Beverages, Bubble Tea': 'Beverages',
    'British, Pub Food': 'Pub',
    'Bubble Tea': 'Beverages',
    'Bubble Tea, Asian': 'Beverages',
    'Bubble Tea, Asian, Tea': 'Beverages',
    'Bubble Tea, Beverages': 'Beverages',
    'Bubble Tea, Desserts': 'Beverages',
    'Bubble Tea, Taiwanese': 'Beverages',
    'Bubble Tea, Tea': 'Beverages',
    'Bubble Tea, Tea, Beverages': 'Beverages',
    'Burger': 'Fast Food',
    'Burger, Bar Food': 'Bar',
    'Burger, Desserts': 'Fast Food',
    'Cafe Food': 'Cafe',
    'Cafe Food, Bar Food': 'Cafe',
    'Cafe Food, Coffee and Tea': 'Cafe',
    'Cafe Food, Coffee and Tea, Desserts': 'Cafe',
    'Cafe Food, Coffee and Tea, Sandwich': 'Cafe',
    'Cafe Food, Desserts, Coffee and Tea': 'Cafe',
    'Cafe Food, Italian': 'Cafe',
    'Cafe Food, Salad, Coffee and Tea': 'Cafe',
    'Chinese': 'Restaurant',
    'Chinese, Asian': 'Restaurant',
    'Chinese, Asian, Dumplings': 'Restaurant',
    'Chinese, Asian, Fast Food': 'Restaurant',
    'Chinese, Asian, Yum Cha': 'Restaurant',
    'Chinese, BBQ': 'Restaurant',
    'Chinese, Coffee and Tea': 'Cafe',
    'Chinese, Contemporary, Asian': 'Restaurant',
    'Chinese, Dumplings': 'Restaurant',
    'Chinese, Hot Pot': 'Restaurant',
    'Chinese, Hot Pot, Sichuan': 'Restaurant',
    'Chinese, Malatang': 'Restaurant',
    'Chinese, Malaysian': 'Restaurant',
    'Chinese, Sichuan': 'Restaurant',
    'Chinese, Taiwanese': 'Restaurant',
    'Coffee and Tea': 'Cafe',
    'Coffee and Tea, Bakery, Meat Pie': 'Cafe',
    'Coffee and Tea, Beverages': 'Cafe',
    'Coffee and Tea, Cafe Food': 'Cafe',
    'Coffee and Tea, Cafe Food, Burger': 'Cafe',
    'Coffee and Tea, Desserts': 'Cafe',
    'Coffee and Tea, Pizza, Cafe Food': 'Cafe',
    'Coffee and Tea, Sandwich': 'Cafe',
    'Coffee and Tea, Sandwich, Cafe Food': 'Cafe',
    'Coffee and Tea, Sandwich, Desserts': 'Cafe',
    'Colombian, Cafe Food': 'Cafe',
    'Contemporary, Italian, Australian': 'Restaurant',
    'Continental, Coffee and Tea, Cafe Food': 'Cafe',
    'Desserts, Beverages, Vietnamese': 'Desserts',
    'Desserts, Cafe Food': 'Desserts',
    'Desserts, Coffee and Tea': 'Desserts',
    'Desserts, Coffee and Tea, Cafe Food': 'Desserts',
    'Desserts, French, Coffee and Tea': 'Desserts',
    'Desserts, Frozen Yogurt': 'Desserts',
    'Desserts, Ice Cream': 'Desserts',
    'Drinks Only': 'Beverages',
    'Dumplings, Japanese, Sushi': 'Restaurant',
    'Dutch, Desserts': 'Desserts',
    'European': 'Restaurant',
    'European, Coffee and Tea': 'Cafe',
    'European, Contemporary': 'Restaurant',
    'European, Desserts, Coffee and Tea': 'Cafe',
    'European, Italian': 'Restaurant',
    'Fast Food, Fried Chicken': 'Fast Food',
    'Fast Food, Japanese, Sushi': 'Fast Food',
    'Fast Food, Pizza': 'Fast Food',
    'Fast Food, Taiwanese, Fried Chicken': 'Fast Food',
    'French': 'Restaurant',
    'French, Crepes': 'Restaurant',
    'French, Crepes, Coffee and Tea, Cafe Food': 'Cafe',
    'French, Desserts, Coffee and Tea': 'Cafe',
    'Fried Chicken, Street Food, Korean': 'Restaurant',
    'Greek': 'Restaurant',
    'Greek, Street Food': 'Restaurant',
    'Healthy Food, Asian, Vegetarian': 'Restaurant',
    'Healthy Food, Cafe Food, Coffee and Tea, Sandwich': 'Cafe',
    'Healthy Food, Coffee and Tea, Cafe Food': 'Cafe',
    'Hot Pot, Chinese': 'Restaurant',
    'Hot Pot, Chinese, Asian': 'Restaurant',
    'Ice Cream': 'Desserts',
    'Ice Cream, Cafe Food': 'Desserts',
    'Ice Cream, Desserts': 'Desserts',
    'Indian': 'Restaurant',
    'Indian, Healthy Food, Vegetarian': 'Restaurant',
    'Indian, South Indian': 'Restaurant',
    'Indonesian': 'Restaurant',
    'Italian': 'Restaurant',
    'Italian, Coffee and Tea': 'Cafe',
    'Italian, Coffee and Tea, Cafe Food': 'Cafe',
    'Italian, Pizza': 'Restaurant',
    'Japanese': 'Restaurant',
    'Japanese, Bubble Tea': 'Beverages',
    'Japanese, Cafe Food, Coffee and Tea': 'Cafe',
    'Japanese, Desserts, Coffee and Tea': 'Cafe',
    'Japanese, Fusion': 'Restaurant',
    'Japanese, Fusion, Asian': 'Restaurant',
    'Japanese, Korean': 'Restaurant',
    'Japanese, Ramen': 'Restaurant',
    'Japanese, Sandwich, Coffee and Tea': 'Cafe',
    'Japanese, Sushi': 'Restaurant',
    'Juices, Beverages': 'Beverages',
    'Juices, Coffee and Tea, Cafe Food': 'Beverages',
    'Juices, Healthy Food': 'Beverages',
    'Kebab, Bakery': 'Fast Food',
    'Kebab, Turkish': 'Fast Food',
    'Korean': 'Restaurant',
    'Korean BBQ, Korean': 'Restaurant',
    'Korean BBQ, Korean, Asian': 'Restaurant',
    'Korean, Asian': 'Restaurant',
    'Korean, Burger, Bar Food, American': 'Bar',
    'Korean, Fast Food': 'Fast Food',
    'Korean, Fried Chicken': 'Restaurant',
    'Korean, Fried Chicken, Asian': 'Restaurant',
    'Korean, Korean BBQ': 'Restaurant',
    'Latin American, Mexican': 'Restaurant',
    'Malaysian': 'Restaurant',
    'Mediterranean': 'Restaurant',
    'Mediterranean, Cafe Food, Coffee and Tea': 'Cafe',
    'Mediterranean, Coffee and Tea, Cafe Food': 'Cafe',
    'Mediterranean, Modern Australian': 'Restaurant',
    'Mexican': 'Restaurant',
    'Middle Eastern': 'Restaurant',
    'Modern Australian': 'Restaurant',
    'Modern Australian, Bar Food': 'Bar',
    'Modern Australian, Coffee and Tea, Cafe Food': 'Cafe',
    'Modern Australian, Fusion, Asian': 'Restaurant',
    'Modern Australian, Steak, Grill': 'Restaurant',
    'Modern European': 'Restaurant',
    'Modern European, Bar Food': 'Bar',
    'Modern European, Contemporary': 'Restaurant',
    'Nepalese': 'Restaurant',
    'Pizza': 'Restaurant',
    'Pizza, Fast Food': 'Fast Food',
    'Pizza, Italian': 'Restaurant',
    'Pizza, Kebab, Fast Food': 'Fast Food',
    'Pizza, Pub Food': 'Pub',
    'Pizza, Sandwich': 'Fast Food',
    'Poké': 'Fast Food',
    'Poké, Healthy Food, Fusion': 'Fast Food',
    'Pub Food': 'Pub',
    'Pub Food, Irish': 'Pub',
    'Pub Food, Modern Australian': 'Pub',
    'Salad, Cafe Food': 'Cafe',
    'Sandwich': 'Cafe',
    'Sandwich, Cafe Food': 'Cafe',
    'Sandwich, Coffee and Tea': 'Cafe',
    'Sandwich, Coffee and Tea, Cafe Food': 'Cafe',
    'Sandwich, Drinks Only': 'Cafe',
    'Spanish, Tapas': 'Restaurant',
    'Steak, Modern Australian': 'Restaurant',
    'Sushi, Japanese': 'Restaurant',
    'Thai': 'Restaurant',
    'Thai, Asian': 'Restaurant',
    'Vegetarian, Coffee and Tea, Cafe Food': 'Cafe',
    'Vietnamese': 'Restaurant',
    'Vietnamese, Healthy Food': 'Restaurant',
    'Vietnamese, Indonesian, Malaysian, Asian': 'Restaurant',
    'Vietnamese, Pho': 'Restaurant' })
z_clean_reduced.Type.value_counts()

Restaurant                                    154
Cafe                                          102
Beverages                                      27
Desserts                                       20
Fast Food                                      16
Bar                                            13
Pub                                             9
Bakery                                          8
Chinese, Hot Pot, Asian, BBQ, Cabinet Food      1
Japanese, Italian                               1
Ice Cream, Asian                                1
Middle Eastern, Kebab                           1
American                                        1
Bakery, Turkish                                 1
Ice Cream, Desserts, Coffee and Tea             1
Tea                                             1
Desserts, Taiwanese                             1
Kebab                                           1
Kebab, Iranian                                  1
Korean, Asian, Korean BBQ, Fried Chicken        1


In [127]:
z_clean_reduced = z_clean_reduced.loc[z_clean_reduced["Aggregate Rating"] > 0,:]
list(z_clean_reduced.Type.sort_values().unique())

['Bakery',
 'Bakery, Turkish',
 'Bar',
 'Beverages',
 'Beverages, Healthy Food, Salad',
 'Burger, Fast Food',
 'Cafe',
 'Chinese, Hot Pot, Asian, BBQ, Cabinet Food',
 'Chinese, Street Food',
 'Desserts',
 'Desserts, Bakery',
 'Desserts, Taiwanese',
 'Fast Food',
 'Fast Food, Asian',
 'Fast Food, German, Eastern European',
 'Fish and Chips',
 'Healthy Food',
 'Ice Cream, Asian',
 'Ice Cream, Desserts, Coffee and Tea',
 'Japanese, Asian',
 'Japanese, Italian',
 'Kebab, Iranian',
 'Korean, Asian, Korean BBQ, Fried Chicken',
 'Malatang',
 'Middle Eastern, Kebab',
 'Pub',
 'Restaurant',
 'Tea']

In [128]:
z_clean_reduced = z_clean_reduced.replace({
    'Bakery, Turkish': 'Bakery',
    'Beverages, Healthy Food, Salad': 'Fast Food',
    'Burger, Fast Food': 'Fast Food',
    'Chinese, Hot Pot, Asian, BBQ, Cabinet Food': 'Restaurant',
    'Chinese, Street Food': 'Restaurant',
    'Desserts, Bakery': 'Desserts',
    'Desserts, Taiwanese': 'Desserts',
    'Fast Food, Asian': 'Fast Food',
    'Fast Food, German, Eastern European': 'Fast Food',
    'Fish and Chips': 'Fast Food',
    'Healthy Food': 'Fast Food',
    'Ice Cream, Asian': 'Desserts',
    'Ice Cream, Desserts, Coffee and Tea': 'Desserts',
    'Japanese, Asian': 'Restaurant',
    'Japanese, Italian': 'Restaurant',
    'Kebab, Iranian': 'Restaurant',
    'Korean, Asian, Korean BBQ, Fried Chicken': 'Restaurant',
    'Malatang': 'Restaurant',
    'Middle Eastern, Kebab': 'Restaurant',
    'Tea': 'Beverages' })
z_clean_reduced.Type.value_counts()

Restaurant    151
Cafe           84
Desserts       24
Beverages      22
Fast Food      19
Bar            12
Pub             9
Bakery          8
Name: Type, dtype: int64

In [129]:
# Save Zomato DataFrame (Unconsolidated Types) to CSV
z_clean_csv = z_clean.to_csv("Zomato_Restaurants_All.csv")
# Save Zomato DataFrame (Consolidated Types)
z_clean_types_csv = z_clean_reduced.to_csv("Zomato_Restaurants_All_Consolidated-Types.csv")

In [10]:
# Consolidate eatery types 
z_clean_2 = z_clean.replace({
    'American, Bar Food':'Bar Food',
    'American, Coffee and Tea, Cafe Food': 'Cafe Food',
    'American, Fried Chicken': 'American',
    'American, Pub Food': 'Pub Food',
    'American, Steak':'American',   
    'Asian, Desserts': 'Desserts',
    'Asian, Desserts, Ice Cream': 'Desserts',
    'Asian, Fast Food':'Fast Food',
    'Asian, Fusion': 'Asian',
    'Asian, Indonesian':'Indonesian',
    'Asian, Japanese':'Japanese',
    'Asian, Malaysian, Chinese':'Malaysian',
    'Asian, Sri Lankan, Modern Australian':'Asian',
    'Australian, Contemporary': 'Australian',
    'BBQ, Brazilian, Steak': 'BBQ',
    'BBQ, Pizza, Fast Food': 'BBQ',
    'BBQ, Steak': 'BBQ',
    'Bakery, Coffee and Tea':'Bakery',
    'Bakery, Desserts':'Desserts',
    'Bakery, Fast Food':'Fast Food',
    'Bakery, French':'Bakery',
    'Bakery, Middle Eastern':'Bakery',
    'Bakery, Patisserie':'Bakery',
    'Bakery, Turkish':'Bakery',
    'Bar Food, Asian Fusion':'Bar Food',
    'Bar Food, Asian, Dumplings':'Bar Food',
    'Bar Food, Contemporary':'Bar Food',
    'Bar Food, French':'Bar Food',
    'Bar Food, Middle Eastern':'Bar Food',
    'Bar Food, Modern Australian':'Bar Food',
    'Bar Food, Modern European':'Bar Food',
    'Beverages, Bubble Tea, Tea':'Bubble Tea',
    'Bubble Tea, Asian':'Bubble Tea',
    'Bubble Tea, Cafe Food, Coffee and Tea':'Bubble Tea',
    'Bubble Tea, Desserts':'Bubble Tea',
    'Bubble Tea, Taiwanese':'Bubble Tea',
    'Bubble Tea, Tea, Beverages':'Bubble Tea',
    'Bubble Tea, Tea, Taiwanese, Beverages':'Bubble Tea',
    'Burger, Bar Food':'Burger',
    'Burger, Fast Food':'Fast Food',
    'Cafe Food, Bar Food':'Cafe Food',
    'Cafe Food, Coffee and Tea':'Cafe Food',
    'Cafe Food, Coffee and Tea, Mediterranean':'Cafe Food',
    'Cafe Food, Coffee and Tea, Modern Australian':'Cafe Food',
    'Cafe Food, Coffee and Tea, Vietnamese':'Cafe Food',
    'Cafe Food, Desserts, Coffee and Tea':'Cafe Food',
    'Cafe Food, Korean':'Cafe Food',
    'Chinese, Asian':'Chinese',
    'Chinese, BBQ':'Chinese',
    'Chinese, Cantonese, Coffee and Tea':'Chinese',
    'Chinese, Coffee and Tea':'Chinese',
    'Chinese, Dumplings':'Chinese',
    'Chinese, Dumplings, Hot Pot':'Chinese',
    'Chinese, Fast Food':'Fast Food',
    'Chinese, Hot Pot, Asian, BBQ, Cabinet Food':'Chinese',
    'Chinese, Hot Pot, Sichuan':'Chinese',
    'Chinese, Malatang':'Chinese',
    'Chinese, Malaysian':'Malaysian',
    'Chinese, Seafood':'Chinese',
    'Chinese, Sichuan':'Chinese',
    'Chinese, Street Food':'Chinese',
    'Chinese, Taiwanese':'Chinese',
    'Coffee and Tea':'Cafe Food',
    'Coffee and Tea, American':'Cafe Food',
    'Coffee and Tea, Beverages':'Cafe Food',
    'Coffee and Tea, Bubble Tea, Poké':'Bubble Tea',
    'Coffee and Tea, Cafe Food':'Cafe Food',
    'Coffee and Tea, Cafe Food, Sandwich':'Cafe Food',
    'Coffee and Tea, Desserts':'Cafe Food',
    'Coffee and Tea, Desserts, Frozen Yogurt':'Cafe Food',
    'Coffee and Tea, Healthy Food, Cafe Food':'Cafe Food',
    'Coffee and Tea, Healthy Food, Salad, Sandwich':'Cafe Food',
    'Coffee and Tea, Healthy Food, Sandwich':'Cafe Food',
    'Coffee and Tea, Italian':'Cafe Food',
    'Coffee and Tea, Modern Australian':'Cafe Food',
    'Coffee and Tea, Pizza, Cafe Food':'Cafe Food',
    'Coffee and Tea, Sandwich':'Cafe Food',
    'Coffee and Tea, Sandwich, Cafe Food':'Cafe Food',
    'Coffee and Tea, Sandwich, Desserts':'Cafe Food',
    'Coffee and Tea, Turkish':'Cafe Food',
    'Continental, Australian':'Australian',
    'Continental, Coffee and Tea, Cafe Food':'Cafe Food',
    'Deli, Coffee and Tea':'Cafe Food',
    'Desserts, Bakery':'Desserts',
    'Desserts, Beverages, Vietnamese':'Desserts',
    'Desserts, Cafe Food':'Desserts',
    'Desserts, Coffee and Tea':'Desserts',
    'Desserts, Coffee and Tea, Cafe Food':'Desserts',
    'Desserts, French, Coffee and Tea':'Desserts',
    'Desserts, Ice Cream':'Desserts',
    'Desserts, Modern Australian':'Desserts',
    'Desserts, Taiwanese':'Desserts',
    'Dumplings':'Chinese',
    'Dumplings, Chinese':'Chinese',
    'European, Bar Food':'Bar Food',
    'European, Modern Australian':'European',
    'European, Swiss':'European',
    'Fast Food, Fried Chicken':'Fast Food',
    'Fast Food, German, Eastern European':'Fast Food',
    'Fast Food, Japanese, Sushi':'Fast Food',
    'Fast Food, Taiwanese, Fried Chicken':'Fast Food',
    'French, Desserts, Coffee and Tea':'French',
    'Fried Chicken, Korean, Asian':'Korean',
    'Frozen Yogurt, Desserts':'Desserts',
    'Healthy Food, Cafe Food, Coffee and Tea, Sandwich':'Cafe Food',
    'Healthy Food, Fast Food, Salad':'Healthy Food',
    'Healthy Food, Juices':'Drinks Only',
    'Healthy Food, Salad, Sandwich':'Healthy Food',
    'Healthy Food, Vegetarian, Salad':'Healthy Food',
    'Hot Pot, Chinese, Asian':'Chinese',
    'Hot Pot, Sichuan, Chinese':'Chinese',
    'Ice Cream, Desserts, Coffee and Tea':'Desserts',
    'Indian, Sri Lankan':'Indian',
    'Indian, Vegetarian, Vegan':'Indian',
    'Indonesian, Malaysian':'Indonesian',
    'Italian, Bar Food':'Bar Food',
    'Italian, Cafe Food': 'Cafe Food',
    'Italian, Coffee and Tea, Cafe Food': 'Cafe Food',
    'Italian, European':'Italian',
    'Italian, Mediterranean':'Italian',
    'Italian, Pizza':'Italian',
    'Japanese, Australian, Asian':'Japanese',
    'Japanese, Beverages, Sushi':'Japanese',
    'Japanese, Cafe Food, Coffee and Tea':'Cafe Food',
    'Japanese, Desserts, Tea':'Desserts',
    'Japanese, Italian':'Japanese',
    'Japanese, Ramen':'Japanese',
    'Japanese, Sushi':'Japanese',
    'Japanese, Teppanyaki':'Japanese',
    'Juices, Beverages':'Drinks Only',
    'Juices, Cafe Food':'Drinks Only',
    'Kebab, Coffee and Tea':'Fast Food',
    'Korean BBQ, Korean':'Korean',
    'Korean, Asian':'Korean',
    'Korean, Chinese':'Korean',
    'Korean, Fried Chicken':'Korean',
    'Korean, Healthy Food, Fast Food, Asian':'Fast Food',
    'Korean, Korean BBQ':'Korean',
    'Malaysian, Coffee and Tea':'Cafe Food',
    'Modern Australian':'Australian',
    'Modern Australian, Burger, Bar Food':'Burger',
    'Modern Australian, Coffee and Tea':'Cafe Food',
    'Modern Australian, Coffee and Tea, Cafe Food': 'Cafe Food',
    'Modern Australian, Fusion, Asian':'Australian',
    'Modern Australian, Grill':'Australian',
    'Modern European':'European',
    'Modern European, Modern Australian':'European',
    'Patisserie':'Bakery',
    'Pho, Vietnamese, Asian':'Vietnamese',
    'Pizza, Italian':'Italian',
    'Pub Food, Modern Australian':'Pub Food',
    'Sandwich':'Cafe Food',
    'Sandwich, Cafe Food, Coffee and Tea':'Cafe Food',
    'Sandwich, Coffee and Tea':'Cafe Food',
    'Sandwich, Coffee and Tea, Cafe Food':'Cafe Food',
    'Sandwich, Fast Food':'Cafe Food',
    'Sandwich, Healthy Food, Coffee and Tea':'Cafe Food',
    'Sandwich, Vietnamese':'Vietnamese',
    'Seafood, Grill':'Seafood',
    'Seafood, Japanese':'Seafood',
    'Spanish, Bar Food, Cafe Food':'Cafe Food',
    'Spanish, Tapas':'European',
    'Sushi':'Japanese',
    'Sushi, Japanese, Korean':'Japanese',
    'Tea':'Drinks Only',
    'Tea, Bubble Tea, Beverages':'Bubble Tea',
    'Thai, Asian':'Thai',
    'Thai, Asian Fusion':'Thai',
    'Thai, BBQ, Hot Pot':'Thai',
    'Vegetarian, Coffee and Tea, Cafe Food':'Cafe Food',
    'Vietnamese, Asian, Fusion':'Vietnamese',
    'Vietnamese, Healthy Food':'Vietnamese',
    'Vietnamese, Pho':'Vietnamese',
    'Vietnamese, Pho, Asian':'Vietnamese',
    'Vietnamese, Sandwich':'Vietnamese'
    
})
list(z_clean_2.Type.sort_values().unique())

['American',
 'Asian',
 'Australian',
 'BBQ',
 'Bakery',
 'Bar Food',
 'Bubble Tea',
 'Burger',
 'Cafe Food',
 'Chinese',
 'Contemporary',
 'Desserts',
 'Drinks Only',
 'European',
 'Fast Food',
 'Fish and Chips',
 'French',
 'Hawaiian, Sushi',
 'Healthy Food',
 'Indian',
 'Indonesian',
 'Italian',
 'Japanese',
 'Korean',
 'Latin American, Peruvian',
 'Malatang',
 'Malaysian',
 'Meat Pie',
 'Mediterranean',
 'Mexican',
 'Middle Eastern',
 'Nepalese',
 'Portuguese',
 'Pub Food',
 'Seafood',
 'Sri Lankan',
 'Thai',
 'Vietnamese']

In [ ]:
# get coordinates of all relevant pedestrian traffic scanners

# add new column and bin data based on proximity to a sensor


In [ ]:
# Map locations of each restaurant onto Google Maps

# Compare with foot traffic

    # Make heatmap of foot traffic to see traffic volume

    # Assign ranking to foot traffic volume and add it as a numerical value to the Zomato DF as a new column
    
    # do a groupby